In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import pickle
import pandas as pd
import numpy as np
import os
import pickle
import logging
from collections import defaultdict
from tqdm import tqdm  # Import tqdm for the progress bar
import numpy as np
import wandb
import torch
import torch.nn.functional as F


from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [16]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


class BaseEntailment:
    def save_prediction_cache(self):
        pass


class EntailmentDeberta(BaseEntailment):
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v2-xlarge-mnli")
        self.model = AutoModelForSequenceClassification.from_pretrained(
            "microsoft/deberta-v2-xlarge-mnli").to(DEVICE)

    def check_implication(self, text1, text2, *args, **kwargs):
        inputs = self.tokenizer(text1, text2, return_tensors="pt").to(DEVICE)
        # The model checks if text1 -> text2, i.e. if text2 follows from text1.
        # check_implication('The weather is good', 'The weather is good and I like you') --> 1
        # check_implication('The weather is good and I like you', 'The weather is good') --> 2
        outputs = self.model(**inputs)
        logits = outputs.logits
        # Deberta-mnli returns `neutral` and `entailment` classes at indices 1 and 2.
        largest_index = torch.argmax(F.softmax(logits, dim=1))  # pylint: disable=no-member
        prediction = largest_index.cpu().item()
        if os.environ.get('DEBERTA_FULL_LOG', False):
            logging.info('Deberta Input: %s -> %s', text1, text2)
            logging.info('Deberta Prediction: %s', prediction)

        return prediction


class EntailmentLLM(BaseEntailment):

    entailment_file = 'entailment_cache.pkl'

    def __init__(self, entailment_cache_id, entailment_cache_only):
        self.prediction_cache = self.init_prediction_cache(entailment_cache_id)
        self.entailment_cache_only = entailment_cache_only

    def init_prediction_cache(self, entailment_cache_id):
        if entailment_cache_id is None:
            return dict()

        logging.info('Restoring prediction cache from %s', entailment_cache_id)

        api = wandb.Api()
        run = api.run(entailment_cache_id)
        run.file(self.entailment_file).download(
            replace=True, exist_ok=False, root=wandb.run.dir)

        with open(f'{wandb.run.dir}/{self.entailment_file}', "rb") as infile:
            return pickle.load(infile)

    def save_prediction_cache(self):
        # Write the dictionary to a pickle file.
        utils.save(self.prediction_cache, self.entailment_file)

    def check_implication(self, text1, text2, example=None):
        if example is None:
            raise ValueError
        prompt = self.equivalence_prompt(text1, text2, example['question'])

        logging.info('%s input: %s', self.name, prompt)

        hashed = oai.md5hash(prompt)
        if hashed in self.prediction_cache:
            logging.info('Restoring hashed instead of predicting with model.')
            response = self.prediction_cache[hashed]
        else:
            if self.entailment_cache_only:
                raise ValueError
            response = self.predict(prompt, temperature=0.02)
            self.prediction_cache[hashed] = response

        logging.info('%s prediction: %s', self.name, response)

        binary_response = response.lower()[:30]
        if 'entailment' in binary_response:
            return 2
        elif 'neutral' in binary_response:
            return 1
        elif 'contradiction' in binary_response:
            return 0
        else:
            logging.warning('MANUAL NEUTRAL!')
            return 1


class EntailmentGPT4(EntailmentLLM):

    def __init__(self, entailment_cache_id, entailment_cache_only):
        super().__init__(entailment_cache_id, entailment_cache_only)
        self.name = 'gpt-4'

    def equivalence_prompt(self, text1, text2, question):

        prompt = f"""We are evaluating answers to the question \"{question}\"\n"""
        prompt += "Here are two possible answers:\n"
        prompt += f"Possible Answer 1: {text1}\nPossible Answer 2: {text2}\n"
        prompt += "Does Possible Answer 1 semantically entail Possible Answer 2? Respond with entailment, contradiction, or neutral."""

        return prompt

    def predict(self, prompt, temperature):
        return oai.predict(prompt, temperature, model=self.name)


class EntailmentGPT35(EntailmentGPT4):

    def __init__(self, entailment_cache_id, entailment_cache_only):
        super().__init__(entailment_cache_id, entailment_cache_only)
        self.name = 'gpt-3.5'


class EntailmentGPT4Turbo(EntailmentGPT4):

    def __init__(self, entailment_cache_id, entailment_cache_only):
        super().__init__(entailment_cache_id, entailment_cache_only)
        self.name = 'gpt-4-turbo'


class EntailmentLlama(EntailmentLLM):

    def __init__(self, entailment_cache_id, entailment_cache_only, name):
        super().__init__(entailment_cache_id, entailment_cache_only)
        self.name = name
        self.model = HuggingfaceModel(
            name, stop_sequences='default', max_new_tokens=30)

    def equivalence_prompt(self, text1, text2, question):

        prompt = f"""We are evaluating answers to the question \"{question}\"\n"""
        prompt += "Here are two possible answers:\n"
        prompt += f"Possible Answer 1: {text1}\nPossible Answer 2: {text2}\n"
        prompt += "Does Possible Answer 1 semantically entail Possible Answer 2? Respond only with entailment, contradiction, or neutral.\n"""
        prompt += "Response:"""

        return prompt

    def predict(self, prompt, temperature):
        predicted_answer, _, _ = self.model.predict(prompt, temperature)
        return predicted_answer


def context_entails_response(context, responses, model):
    votes = []
    for response in responses:
        votes.append(model.check_implication(context, response))
    return 2 - np.mean(votes)


def get_semantic_ids(strings_list, model, strict_entailment=False, example=None):
    """Group list of predictions into semantic meaning."""

    def are_equivalent(text1, text2):

        implication_1 = model.check_implication(text1, text2, example=example)
        implication_2 = model.check_implication(text2, text1, example=example)  # pylint: disable=arguments-out-of-order
        assert (implication_1 in [0, 1, 2]) and (implication_2 in [0, 1, 2])

        if strict_entailment:
            semantically_equivalent = (implication_1 == 2) and (implication_2 == 2)

        else:
            implications = [implication_1, implication_2]
            # Check if none of the implications are 0 (contradiction) and not both of them are neutral.
            semantically_equivalent = (0 not in implications) and ([1, 1] != implications)

        return semantically_equivalent

    # Initialise all ids with -1.
    semantic_set_ids = [-1] * len(strings_list)
    # Keep track of current id.
    next_id = 0
    for i, string1 in enumerate(strings_list):
        # Check if string1 already has an id assigned.
        if semantic_set_ids[i] == -1:
            # If string1 has not been assigned an id, assign it next_id.
            semantic_set_ids[i] = next_id
            for j in range(i+1, len(strings_list)):
                # Search through all remaining strings. If they are equivalent to string1, assign them the same id.
                if are_equivalent(string1, strings_list[j]):
                    semantic_set_ids[j] = next_id
            next_id += 1

    assert -1 not in semantic_set_ids

    return semantic_set_ids


def logsumexp_by_id(semantic_ids, log_likelihoods, agg='sum_normalized'):
    """Sum probabilities with the same semantic id.

    Log-Sum-Exp because input and output probabilities in log space.
    """
    unique_ids = sorted(list(set(semantic_ids)))
    assert unique_ids == list(range(len(unique_ids)))
    log_likelihood_per_semantic_id = []

    for uid in unique_ids:
        # Find positions in `semantic_ids` which belong to the active `uid`.
        id_indices = [pos for pos, x in enumerate(semantic_ids) if x == uid]
        # Gather log likelihoods at these indices.
        id_log_likelihoods = [log_likelihoods[i] for i in id_indices]
        if agg == 'sum_normalized':
            # log_lik_norm = id_log_likelihoods - np.prod(log_likelihoods)
            log_lik_norm = id_log_likelihoods - np.log(np.sum(np.exp(log_likelihoods)))
            logsumexp_value = np.log(np.sum(np.exp(log_lik_norm)))
        else:
            raise ValueError
        log_likelihood_per_semantic_id.append(logsumexp_value)

    return log_likelihood_per_semantic_id


def predictive_entropy(log_probs):
    """Compute MC estimate of entropy.

    `E[-log p(x)] ~= -1/N sum_i log p(x_i)`, i.e. the average token likelihood.
    """

    entropy = -np.sum(log_probs) / len(log_probs)

    return entropy


def predictive_entropy_rao(log_probs):
    entropy = -np.sum(np.exp(log_probs) * log_probs)
    return entropy


def cluster_assignment_entropy(semantic_ids):
    """Estimate semantic uncertainty from how often different clusters get assigned.

    We estimate the categorical distribution over cluster assignments from the
    semantic ids. The uncertainty is then given by the entropy of that
    distribution. This estimate does not use token likelihoods, it relies soley
    on the cluster assignments. If probability mass is spread of between many
    clusters, entropy is larger. If probability mass is concentrated on a few
    clusters, entropy is small.

    Input:
        semantic_ids: List of semantic ids, e.g. [0, 1, 2, 1].
    Output:
        cluster_entropy: Entropy, e.g. (-p log p).sum() for p = [1/4, 2/4, 1/4].
    """

    n_generations = len(semantic_ids)
    counts = np.bincount(semantic_ids)
    probabilities = counts/n_generations
    assert np.isclose(probabilities.sum(), 1)
    entropy = - (probabilities * np.log(probabilities)).sum()
    return entropy


In [17]:
def read_pickle_file_into_dict(pickle_address):
    """
    Transforms a dictionary with 'SE' key such that:
    - The key of dict_old becomes an entry under the "question" key in dict_new.
    - The tuples in the nested list under 'SE' are split into "tokens", "softmax_probability", and "logits".
    - Converts the nested list of tokens into sentences and adds them under a new key "responses" as a nested list.
    - Calculates log likelihoods and adds them under a new key "LogLikelihoods".

    Args:
        pickle_address (str): The file path to the pickle file.

    Returns:
        dict: The transformed dictionary with multiple top-level keys.

    Raises:
        ValueError: If the lengths of the top-level keys in the transformed dictionary are not the same,
                   or if the lengths of tokens, softmax probabilities, logits, responses, and LogLikelihoods within the nested lists do not match.
        FileNotFoundError: If the pickle file cannot be found or read.
    """
    try:
        # Attempt to load the pickle file
        dict_old = pickle.load(open(pickle_address, 'rb'))
    except (FileNotFoundError, IOError) as e:
        raise FileNotFoundError(f"Error reading the pickle file. The location provided is incorrect: {pickle_address}") from e
    except Exception as e:
        raise Exception(f"An unexpected error occurred while reading the pickle file: {e}") from e

    # Initialize the new dictionary with multiple keys
    dict_new = {
        "questions": [],
        "tokens": [],
        "softmax_probability": [],
        "logits": [],
        "responses": [],  # New key to store sentences as a nested list
        "LogLikelihoods": []  # New key to store log likelihoods
    }

    try:
        # Iterate over dict_old to transform it
        for question, data in dict_old.items():
            # Add the question to the "questions" key
            dict_new["questions"].append(question)

            # Initialize lists to store tokens, probabilities, logits, sentences, and log likelihoods for this question
            question_tokens = []
            question_softmax_probs = []
            question_logits = []
            question_responses = []  # Nested list to store sentences for this question
            question_log_likelihoods = []  # Nested list to store log likelihoods

            # Process the nested list of tuples under 'SE' key
            for inner_list in data['SE']:  # Access the 'SE' key here
                tokens = []
                softmax_probs = []
                logits = []

                for tup in inner_list:
                    tokens.append(tup[0])  # First element of the tuple
                    softmax_probs.append(float(tup[1]))  # Convert numpy float32 to Python float
                    logits.append(float(tup[2]))  # Convert numpy float32 to Python float

                # Sanity check: Ensure the lengths of tokens, softmax probabilities, and logits match
                if len(tokens) != len(softmax_probs) or len(tokens) != len(logits):
                    raise ValueError(
                        f"Sanity check failed: The number of tokens ({len(tokens)}), softmax probabilities "
                        f"({len(softmax_probs)}), and logits ({len(logits)}) do not match for question '{question}'."
                    )

                # Convert tokens into a sentence and add to responses
                sentence = ''.join(token.replace('▁', ' ') for token in tokens if token != '</s>').strip()
                question_responses.append([sentence])  # Add the sentence as a nested list

                # Calculate log likelihoods
                log_likelihoods = [np.log(p) for p in softmax_probs if p > 0]  # Avoid log(0)
                question_log_likelihoods.append(log_likelihoods)  # Add to the nested list

                # Append the processed lists to the respective question-level lists
                question_tokens.append(tokens)
                question_softmax_probs.append(softmax_probs)
                question_logits.append(logits)

            # Sanity check: Ensure the lengths of inner lists for tokens, softmax probabilities, logits, and LogLikelihoods match
            inner_lengths = [
                len(question_tokens),
                len(question_softmax_probs),
                len(question_logits),
                len(question_log_likelihoods)
            ]
            if len(set(inner_lengths)) != 1:
                raise ValueError(
                    f"Sanity check failed: The lengths of inner lists do not match for question '{question}'. "
                    f"Lengths: {inner_lengths}"
                )

            # Add the processed lists to the top-level keys
            dict_new["tokens"].append(question_tokens)
            dict_new["softmax_probability"].append(question_softmax_probs)
            dict_new["logits"].append(question_logits)
            dict_new["responses"].append(question_responses)
            dict_new["LogLikelihoods"].append(question_log_likelihoods)

        # Sanity check: Ensure all top-level keys have the same length
        lengths = [len(dict_new[key]) for key in dict_new]
        if len(set(lengths)) != 1:
            raise ValueError(
                f"Sanity check failed: The lengths of the top-level keys are not the same. "
                f"Lengths: {dict(zip(dict_new.keys(), lengths))}"
            )

    except Exception as e:
        raise Exception(f"An error occurred while transforming the dictionary: {e}") from e

    return dict_new

In [18]:
dict_new = read_pickle_file_into_dict("/content/drive/MyDrive/Projects/Semantic Uncertainty/7b_DD_SE_h100_1 (2).pkl")

In [19]:
print(dict_new.keys())

dict_keys(['questions', 'tokens', 'softmax_probability', 'logits', 'responses', 'LogLikelihoods'])


In [20]:
index_to_print = 10

# Print the value at the specified index for each key
for key in dict_new.keys():
    if index_to_print < len(dict_new[key]):  # Check if the index is valid
        print(f"{key}: {dict_new[key][index_to_print]}")
    else:
        print(f"{key}: Index {index_to_print} is out of range.")

questions: What is the Japanese share index called?
tokens: [['▁', '▁The', '▁Japanese', '▁share', '▁index', '▁is', '▁called', '▁the', '▁Nik', 'ke', 'i', '▁', '2', '2', '5', '.', '</s>'], ['▁', '▁The', '▁Japanese', '▁share', '▁index', '▁is', '▁called', '▁the', '▁Nik', 'ke', 'i', '▁', '2', '2', '5', '.', '</s>'], ['▁', '▁The', '▁Japanese', '▁share', '▁index', '▁is', '▁called', '▁the', '▁Nik', 'ke', 'i', '▁', '2', '2', '5', '.', '</s>'], ['▁', '▁The', '▁Japanese', '▁share', '▁index', '▁is', '▁called', '▁the', '▁Nik', 'ke', 'i', '▁', '2', '2', '5', '.', '</s>'], ['▁', '▁The', '▁Japanese', '▁share', '▁index', '▁is', '▁called', '▁the', '▁Nik', 'ke', 'i', '▁', '2', '2', '5', '.', '</s>'], ['▁', '▁The', '▁Japanese', '▁share', '▁index', '▁is', '▁called', '▁the', '▁Nik', 'ke', 'i', '▁', '2', '2', '5', '.', '</s>'], ['▁', '▁The', '▁Japanese', '▁share', '▁index', '▁is', '▁called', '▁the', '▁Nik', 'ke', 'i', '▁', '2', '2', '5', '.', '</s>'], ['▁', '▁The', '▁Japanese', '▁share', '▁index', '▁is', '▁c

In [21]:
entailment_model = EntailmentDeberta()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/952 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.45M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.77G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.77G [00:00<?, ?B/s]

In [22]:
entropies = defaultdict(list)


In [23]:
result_dict = {'semantic_ids': []}

In [24]:
responses_log_likelihoods = []

# Iterate through each outer list
for outer in dict_new['LogLikelihoods']:
    # Initialize a list to hold the sums for the current outer list
    summed_inner = []

    # Iterate through each inner list
    for inner in outer:
        # Sum the values in the innermost list and append to summed_inner
        summed_inner.append(np.sum(inner))

    # Append the summed inner list to the final result
    responses_log_likelihoods.append(summed_inner)

# Convert the result to a 2D NumPy array
responses_log_likelihoods_arr = np.array(responses_log_likelihoods)


In [25]:
responses_arr = dict_new['responses']


In [26]:
# Wrap the range with tqdm to display the progress bar
for i in tqdm(range(len(responses_arr)), desc="Processing responses"):
    responses = responses_arr[i]
    log_liks = responses_log_likelihoods_arr[i]
    semantic_ids = get_semantic_ids(
                responses, model=entailment_model,
                strict_entailment=True)

    # Store these cluster assignments for each validation sample.
    result_dict['semantic_ids'].append(semantic_ids)

    # Compute an entropy that measures how uncertain the system is about cluster assignment.
    entropies['cluster_assignment_entropy'].append(cluster_assignment_entropy(semantic_ids))

    # Here we compute a "regular entropy" from the average token log-likelihood.
    log_liks_agg = [np.mean(log_lik) for log_lik in log_liks]
    entropies['regular_entropy'].append(predictive_entropy(log_liks_agg))

    # We also compute a "semantic entropy" that groups responses by semantic cluster,
    # summing or normalizing the associated log-likelihoods for each cluster, then computing entropy.
    log_likelihood_per_semantic_id = logsumexp_by_id(semantic_ids, log_liks_agg, agg='sum_normalized')
    pe = predictive_entropy_rao(log_likelihood_per_semantic_id)
    entropies['semantic_entropy'].append(pe)



Processing responses:  84%|████████▍ | 3626/4301 [2:07:47<29:00,  2.58s/it]Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors

Processing responses: 100%|██████████| 4301/4301 [2:33:26<00:00,  2.14s/it]


In [27]:
# Assuming dict_new is the dictionary with the keys ['questions', 'tokens', 'softmax_probability', 'logits', 'responses', 'LogLikelihoods']
# and entropies is the dictionary you want to append

# Perform sanity check
if len(dict_new["questions"]) != len(entropies["cluster_assignment_entropy"]):
    raise ValueError(
        f"Sanity check failed: The length of 'questions' ({len(dict_new['questions'])}) "
        f"does not match the length of 'entropies' ({len(entropies['cluster_assignment_entropy'])})."
    )

# Append entropies to dict_new
dict_new["entropies"] = entropies

# Specify the file name for saving
output_file = "final_data_with_entropies.pkl"

output_dir = "/content/drive/MyDrive/Projects/Semantic Uncertainty"
output_file = os.path.join(output_dir, "13b_SE_h100_2_final_data_with_entropies.pkl")

# Save the updated dictionary to the specified path
with open(output_file, "wb") as f:
    pickle.dump(dict_new, f)

print(f"Dictionary with entropies saved successfully to {output_file}")

Dictionary with entropies saved successfully to /content/drive/MyDrive/Projects/Semantic Uncertainty/13b_SE_h100_2_final_data_with_entropies.pkl


In [29]:
output_dir = "/content/drive/MyDrive/Projects/Semantic Uncertainty"
output_file = os.path.join(output_dir, "7b_DD_SE_h100_1_final_data_with_entropies.pkl")

# Save the updated dictionary to the specified path
with open(output_file, "wb") as f:
    pickle.dump(dict_new, f)

print(f"Dictionary with entropies saved successfully to {output_file}")

Dictionary with entropies saved successfully to /content/drive/MyDrive/Projects/Semantic Uncertainty/7b_DD_SE_h100_1_final_data_with_entropies.pkl
